In [1]:
import time, csv, requests, datetime, random, datetime, os, shutil, glob
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import tzinfo, timedelta, datetime, date
from os import path
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException,TimeoutException,WebDriverException

In [3]:
path = os.path.join(os.getcwd(),"Data")
stockspath = os.path.join(os.getcwd(),"Data")
corporateactionspath = os.path.join(os.getcwd(),"CorporateActions")
print(stockspath)
print(corporateactionspath)
print(path)

C:\Users\venu\Desktop\Stock Market Analysis\Data
C:\Users\venu\Desktop\Stock Market Analysis\CorporateActions
C:\Users\venu\Desktop\Stock Market Analysis\Data


# Bonus Issue

In [4]:
def BonusIssue(data,date_f,date_t,r1,r2):
    print("inside bonus issue -> " + " -------- " + date_f + " " + date_t + " ---------- ratios : " + str(r1) + " " + str(r2))
    now = date_f
    count = 0
    for i in range(0, data.shape[0]):
        now = data.iloc[i].loc['Date']
        if date_f == now:
            if count == 1:
                data.at[i,"Open Price"] = data.at[i,"Open Price"] * (r2/(r1+r2))
                data.at[i,"Low Price"] = data.at[i,"Low Price"] * (r2/(r1+r2))
                data.at[i,"High Price"] = data.at[i,"High Price"] * (r2/(r1+r2))
                data.at[i,"Close Price"] = data.at[i,"Close Price"] * (r2/(r1+r2))
                data.at[i,"WAP"] = data.at[i,"WAP"] * (r2/(r1+r2))
            date_f = data.iloc[i+1].loc["Date"]
            count = 1
            if date_f == date_t:
                #upating the last column
                data.at[data.shape[0]-1,"Open Price"] = data.at[data.shape[0]-1,"Open Price"] * (r2/(r1+r2))
                data.at[data.shape[0]-1,"Low Price"] = data.at[data.shape[0]-1,"Low Price"] * (r2/(r1+r2))
                data.at[data.shape[0]-1,"High Price"] = data.at[data.shape[0]-1,"High Price"] * (r2/(r1+r2))
                data.at[data.shape[0]-1,"Close Price"] = data.at[data.shape[0]-1,"Close Price"] * (r2/(r1+r2))
                data.at[data.shape[0]-1,"WAP"] = data.at[data.shape[0]-1,"WAP"] * (r2/(r1+r2))
                break
    return data

# Stock Split

In [5]:
def StockSplit(data,date_f, date_t,r1,r2):
    print("inside stock split -> " + " -------- " + date_f + " " + date_t + " ---------- ratios : " + str(r1) + " " + str(r2))
    now = date_f
    count = 0
    for i in range(0, data.shape[0]):
        now = data.iloc[i].loc['Date']
        if date_f == now:
            if count == 1:
                data.at[i,"Open Price"] = data.at[i,"Open Price"] * (r1/r2)
                data.at[i,"Low Price"] = data.at[i,"Low Price"] * (r1/r2)
                data.at[i,"High Price"] = data.at[i,"High Price"] * (r1/r2)
                data.at[i,"Close Price"] = data.at[i,"Close Price"] * (r1/r2)
                data.at[i,"WAP"] = data.at[i,"WAP"] * (r1/r2)
            date_f = data.iloc[i+1].loc["Date"]
            count = 1
            if date_f == date_t:
                #updating the last column
                data.at[data.shape[0]-1,"Open Price"] = data.at[data.shape[0]-1,"Open Price"] * (r1/r2)
                data.at[data.shape[0]-1,"Low Price"] = data.at[data.shape[0]-1,"Low Price"] * (r1/r2)
                data.at[data.shape[0]-1,"High Price"] = data.at[data.shape[0]-1,"High Price"] * (r1/r2)
                data.at[data.shape[0]-1,"Close Price"] = data.at[data.shape[0]-1,"Close Price"] * (r1/r2)
                data.at[data.shape[0]-1,"WAP"] = data.at[data.shape[0]-1,"WAP"] * (r1/r2)
                break
    return data

In [6]:
def apply_corporate_actions(stock_data, corporate_data):
    for each in range(0, corporate_data.shape[0]):
        row = corporate_data.iloc[each].loc['Purpose'].split(" ")
        if row[0] == "Bonus":
            ratio = row[2].split(":")
            stock_data = BonusIssue(stock_data, corporate_data.iloc[each].loc["Ex Date"], stock_data.iloc[stock_data.shape[0]-1].loc['Date'], int(ratio[0]), int(ratio[1]))
        elif row[0] == "Stock":
            ratio1 = row[4].split(".")
            ratio2 = row[6].split(".")
            rat1 = ratio1[1].split("/")
            rat2 = ratio2[1].split("/")
            r1 = int(rat1[0])
            r2 = int(rat2[0])
            stock_data = StockSplit(stock_data, corporate_data.iloc[each].loc["Ex Date"], stock_data.iloc[stock_data.shape[0]-1].loc['Date'] ,r2,r1)
    return stock_data